In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import sys
import torch
import numpy as np

import time
sys.path.append('/content/drive/My Drive/Colab Notebooks (1)')

from resnet50 import ResNet50
import keras
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten

# from imagenet_utils import preprocess_input
from keras.applications.resnet50 import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:472: FutureWarn

In [ ]:
# PATH = os.getcwd()
# Define data path
path="/content/drive/My Drive/Colab Notebooks (1)"
import os
data_path = path + '/MyUniqueHelmets(Lalpul)50pc(224x224)'
data_dir_list = os.listdir(data_path)

img_data_list=[]

In [ ]:
# Load the images and preprocess

for dataset in data_dir_list:
  temp = np.array(img_data_list)
  print(temp.shape)
  img_list=os.listdir(data_path+'/'+ dataset)
  print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
  # count = 1
  for img in img_list:
    # print (count)
    # count+=1
    img_path = data_path + '/'+ dataset + '/'+ img 
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    # print('Input image shape:', x.shape)
    img_data_list.append(x)
  
# Convert to required image shape for the model

img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)


(0,)
Loaded the images of dataset-nonhelmet

(679, 1, 224, 224, 3)
Loaded the images of dataset-helmet

(1357, 1, 224, 224, 3)
(1, 1357, 224, 224, 3)
(1357, 224, 224, 3)


In [ ]:
# Temp
# Define the number of classes
num_classes = 2
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

# Assign labels according to length of both classes
labels[0:679]=0
labels[680:1357]=1

names = ['nonhelmet','helmet']
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
# Temp end

# # Define the number of classes
# num_classes = 2
# num_of_samples = img_data.shape[0]
# labels = np.ones((num_of_samples,),dtype='int64')

# labels[0:1009]=0
# labels[1009:2590]=1

# names = ['helmet','nonhelmet']
# # convert class labels to on-hot encoding
# Y = np_utils.to_categorical(labels, num_classes)

# #Shuffle the dataset
# x,y = shuffle(img_data,Y, random_state=2)
# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [ ]:
# Custom_resnet_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

model = ResNet50(include_top=True, weights='imagenet', input_tensor=image_input)
model.summary()

102858752/102853048 [==============================] - 7s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
________________________

In [ ]:
# Make a custom model copy
last_layer = model.get_layer('avg_pool').output
x= Flatten(name='flatten')(last_layer)
out = Dense(num_classes, activation='softmax', name='output_layer')(x)
custom_resnet_model = Model(inputs=image_input,outputs= out)
custom_resnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
# Freeze all layers except last one
for layer in custom_resnet_model.layers[:-1]:
	layer.trainable = False

custom_resnet_model.layers[-1].trainable

True

In [ ]:
# Train model after splitting data
custom_resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

t=time.time()
hist = custom_resnet_model.fit(X_train, y_train, batch_size=32, epochs=12, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_resnet_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Train on 1085 samples, validate on 272 samples
Epoch 1/12
1085/1085 [==============================] - 238s 220ms/step - loss: 0.5152 - accuracy: 0.7438 - val_loss: 0.4248 - val_accuracy: 0.8015
Epoch 2/12
1085/1085 [==============================] - 240s 221ms/step - loss: 0.3003 - accuracy: 0.8728 - val_loss: 0.4373 - val_accuracy: 0.8051
Epoch 3/12
1085/1085 [==============================] - 242s 223ms/step - loss: 0.2351 - accuracy: 0.9069 - val_loss: 0.3519 - val_accuracy: 0.8603
Epoch 4/12
1085/1085 [==============================] - 241s 222ms/step - loss: 0.2238 - accuracy: 0.9171 - val_loss: 0.3660 - val_accuracy: 0.8309
Epoch 5/12
1085/1085 [==============================] - 243s 224ms/step - loss: 0.1891 - accuracy: 0.9327 - val_loss: 0.3761 - val_accuracy: 0.8235
Epoch 6/12
1085/1085 [==============================] - 241s 222ms/step - loss: 0.1867 - accuracy: 0.9217 - val_loss: 0.3525 - val_accuracy: 0.8272
Epoch 7/12
1085/1085 [==============================] - 242s 223m

Testing

In [ ]:
# import pickle

# # # Save to file in the current working directory
# pkl_filename = "/content/drive/My Drive/Colab Notebooks (1)/pickle_model_26April2020.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(custom_resnet_model, file)


y_preds = pickle_model.predict(X_test)

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

# print (y_test)

test= []
predicted= []
for i in range(len(X_test)):
  print("X=%s, Predicted=%s" % (np.argmax(y_test[i]), np.argmax(y_preds[i])))
  test.append(np.argmax(y_test[i]))
  predicted.append(np.argmax(y_preds[i]))

results = confusion_matrix(test, predicted) 

print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(test, predicted) )
print ('Report : ')
print (classification_report(test, predicted))

# confusion_matrix(test, predicted)



X=1, Predicted=0
X=0, Predicted=0
X=1, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=1, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=0, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=1
X=0, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=1
X=0, Predicted=0
X=0, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=1, Predicted=0
X=1, Predicted=0
X=0, Predicted=0
X=0, Predicted=0
X=1, Predicted=0
X=1, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=1, Predicted=1
X=1, Predicted=1
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=1, Predicted=1
X=1, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=1, Predicted=1
X=1, Predicted=0
X=0, Predicted=0
X=1, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=1, Predicted=1
X=1, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=0, Predicted=0
X=0, Predicted=0
X=1, Predicted=0
X=0, Predicted=0
X=0, Predicted=0
X=1, Predicted=1
X=1, Predicted=0
X=1, Predicted

In [ ]:
import pickle

# # Save to file in the current working directory
pkl_filename = "/content/drive/My Drive/Colab Notebooks (1)/pickle_model_24June2020.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(custom_resnet_model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
# # Calculate the accuracy score and predict target values
# score = pickle_model.score(X_test, y_test)
# print("Test score: {0:.2f} %".format(100 * score))

# Ypredict = pickle_model.predict(X_test)

In [ ]:
# from keras_applications.imagenet_utils import decode_predictions 
# label = decode_predictions(Ypredict)

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import pickle


# # Save to file in the current working directory
pkl_filename = "/content/drive/My Drive/Colab Notebooks (1)/pickle_model_24June2020.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(custom_resnet_model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

# import matplotlib.pyplot as plt
import cv2, os
# from PIL import Image, ImageDraw
# from PIL import Image
# from PIL import ImageFont
# from PIL import ImageDraw
import sys
 
# y_preds = pickle_model.predict(X_test)

folder_path = '/content/drive/My Drive/Colab Notebooks (1)/ForBikersVideoFrames(Bikes 50%)'

helmetfolder_write_path = "/content/drive/My Drive/Colab Notebooks (1)/testforvideo/helmetpredicted"
nonhelmetfolder_write_path = "/content/drive/My Drive/Colab Notebooks (1)/testforvideo/nonhelmetpredicted"

fhelmet = open("/content/drive/My Drive/Colab Notebooks (1)/testforvideo/helmetpredicted.txt","w")
fnonhelmet = open("/content/drive/My Drive/Colab Notebooks (1)/testforvideo/nonhelmetpredicted.txt","w")

for imgname in os.listdir(folder_path):
  img_path = os.path.join(folder_path,imgname)
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = pickle_model.predict(x)
  
  prediction = np.argmax(preds[0])
  print(prediction)
  class_labels = ['helmet','nonhelmet']
  # print(class_labels[prediction])
  # plt.imshow(img)
  # plt.show()
  
  # img1 = Image.open(img_path)
  # draw = ImageDraw.Draw(img1)
  # font = ImageFont.truetype("/content/drive/My Drive/Colab Notebooks (1)/micross.ttf", 10)
  # draw.text((0, 0), class_labels[prediction], (255, 0, 0), font=font)


  thisimg = cv2.imread(img_path)
  if class_labels[prediction] == 'helmet':
    fhelmet.write(imgname)
  elif class_labels[prediction] == 'nonhelmet':
    fnonhelmet.write(imgname)


  # thisimg = cv2.imread(img_path)
  # if class_labels[prediction] == 'helmet':
  #   out_file = os.path.join(helmetfolder_write_path,imgname)
  #   cv2.imwrite(out_file, thisimg)
  #   print ("Saved")
  # elif class_labels[prediction] == 'nonhelmet':
  #   out_file = os.path.join(nonhelmetfolder_write_path,imgname)
  #   cv2.imwrite(out_file, thisimg)
  #   print ("Saved")

  # label = decode_predictions(preds, top=1000)
  # label = label[0][:]
  # print(label)

  # lab = decode_predictions(preds)
  # print(lab)
  # print('Predicted:', decode_predictions(preds, top=3)[0])


TypeError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

# print (y_test)

test= []
predicted= []
for i in range(len(X_test)):
  print("X=%s, Predicted=%s" % (np.argmax(y_test[i]), np.argmax(y_preds[i])))
  test.append(np.argmax(y_test[i]))
  predicted.append(np.argmax(y_preds[i]))


# print (np.argmax(y_preds[0]))

In [ ]:
results = confusion_matrix(test, predicted) 

print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(test, predicted) )
print ('Report : ')
print (classification_report(test, predicted))

# confusion_matrix(test, predicted)

Confusion Matrix :
[[170  26]
 [  3 319]]
Accuracy Score : 0.944015444015444
Report : 
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       196
           1       0.92      0.99      0.96       322

    accuracy                           0.94       518
   macro avg       0.95      0.93      0.94       518
weighted avg       0.95      0.94      0.94       518

